# Sprawozdanie z laboratorium 2

***Autor: Adam Dąbkowski***

Celem drugiego laboratorium jest zaimplementowanie algorytmu genetycznego z selekcją ruletkową, krzyżowaniem jednopunktowym oraz sukcesją generacyjną. Dodatkowo należy zbadać wpływ doboru parametrów na przykładzie danego problemu.

## 0. Importowanie niezbędnych bibliotek

In [7]:
import random
import copy
from utils import *

## 1. Tworzenie populacji

In [8]:
class PopulationCreator:
    def __init__(self, population_size, individual_size):
        self._population_size = population_size
        self._individual_size = individual_size

    def create_population(self):
        population = []

        for i in range(self._population_size):
            individual = []
            for j in range(self._individual_size):
                individual.append(random.randint(0, 1))
            population.append(np.array(individual))

        return np.array(population)

## 2. Zdefiniowanie problemu (funkcji celu)

In [9]:
def problem(x):
    h0 = 200
    v0 = 0
    fuel = x.sum()
    m0 = 200 + fuel

    for i in x:
        m0 -= i * 1
        a = (45/m0) * i - 0.09
        v0 = v0 + a
        h0 = h0 + v0

        if (0 <= h0 < 2) and abs(v0) < 2:
            return 2000 - fuel

        elif h0 < 0:
            return -1000 - fuel

    return - fuel

## 3. Implementacja algorytmu genetycznego

Głównym zadaniem drugiego laboratorium jest implementacja algorytmu genetycznego. W tym celu stworzona została klasa ***GeneticAlgorithmSolver***. Podczas tworzenia obiektu tej klasy istnieje możliwość podania czterech parametrów:
- ***t_max*** - liczba iteracji (domyślna wartość wynosi $1000$)
- ***pc*** - prawdopodobieństwo krzyżowania (domyślna wartość wynosi $0,5$)
- ***pm*** - prawdopodobieństwo mutacji (domyślna wartość wynosi $0,5$)
- ***u*** - liczba osobników w populacji (domyślna wartość wynosi $20$)

Klasa ***GeneticAlgorithmSolver*** zawiera także sześć metod:
- ***get_parameters()*** - metoda ta zwraca wartości parametrów
- ***solve()*** - metoda zawierająca w sobie algorytm genetyczny, wykorzystuje metody ***score()*** , ***find_best()*** , ***selection()*** , ***crossing_and_mutation()*** . W wyniku jej wywołania otrzymujemy najlepszego osobnika i wartość funkcji celu
- ***score()*** - metoda zwracająca wyniki dla osobników w populacji
- ***find_best()*** - metoda zwracająca najlepszego osobnika
- ***selection()*** - metoda, która zadaniem jest wybranie lepszych osobników z większym prawdopodobieństwem
- ***crossing_and_mutation()*** - metoda odpowiedzialna za krzyżowanie i mutację

In [24]:
class GeneticAlgorithmSolver:
    def __init__(self, t_max=1000, pc=0.5, pm=0.05, u=250):
        self._t_max = t_max
        self._pc = pc
        self._pm = pm
        self._u = u

    def get_parameters(self):
        return {
            "t_max": self._t_max,
            "pc": self._pc,
            "pm": self._pm,
            "u": self._u
        }

    def solve(self, problem, pop0, printing=False):
        t = 0
        P = pop0
        O = self.score(problem, P)
        x_best, o_best = self.find_best(P, O)
        t_best = 0
        while t < self._t_max:
            S = self.selection(P, O, self._u)
            M = self.crossing_and_mutation(S, self._pc, self._pm, self._u)
            O = self.score(problem, M)
            x, o = self.find_best(M, O)

            if o > o_best:
                x_best = x
                o_best = o
                t_best = t

            P = M
            t += 1

            if printing:
                print(f"Iteration {t}: Best score = {o}, Best score to date = {o_best}")

        if printing:
            print(f" ")
            print(f"Best score = {o_best}, Iteration {t_best}")

        return x_best, o_best

    @staticmethod
    def score(problem, P):
        return [problem(x) for x in P]

    @staticmethod
    def find_best(P, O):
        index = np.argmax(O)
        return P[index], O[index]

    @staticmethod
    def selection(P, O, u):
        O = [o + (1000 + P.shape[1]) for o in O]
        O = [o / sum(O) for o in O]
        indices = np.arange(0, P.shape[0])
        selected_indices = np.random.choice(indices, u, replace=True, p=O)

        return P[selected_indices]

    @staticmethod
    def crossing_and_mutation(S, pc, pm, u):
        M = []

        # crossing
        for i in range(0, u - 1, 2):
            individual_1 = S[i]
            individual_2 = S[i + 1]

            if random.random() < pc:
                crossing_point = random.randint(1, len(individual_1))
                individual_1_crossed = copy.deepcopy(individual_1)
                individual_2_crossed = copy.deepcopy(individual_2)

                individual_1_crossed[crossing_point:] = individual_2[crossing_point:]
                individual_2_crossed[crossing_point:] = individual_1[crossing_point:]

                M.append(individual_1_crossed)
                M.append(individual_2_crossed)

            else:
                M.append(individual_1)
                M.append(individual_2)



        if u % 2 == 1:
            M.append(S[-1])

        # mutation
        for individual in M:
            for i in range(individual.shape[0]):
                if random.random() < pm:
                    individual[i] = 1 if individual[i] == 0 else 0
        return np.array(M)

## 4. Zastosowanie algorytmu genetycznego w celu wyznaczenia najlepszego osobnika

Po zaimplementowaniu algorytmu genetycznego, a także po zdefiniowaniu funkcji celu możemy przejść do testowania solvera w celu znalezienia największej wartości wspomnianej funkcji. Na początku wykorzystany zostanie solver z domyślnymi wartościami, jednakże należy najpierw utworzyć populację początkową. W naszych doświadczeniach liczba osobników w populacji początkowej będzie wynosiła $250$, natomiast wielkość osobnika będzie równa $200$.

In [11]:
population_creator = PopulationCreator(population_size=250, individual_size=200)

In [12]:
pop0 = population_creator.create_population()

Następnie przechodzimy do stworzenia solvera.

In [13]:
solver = GeneticAlgorithmSolver()

Chcąc upewnić się jakie parametry ma nasz solver możemy wykorzystać metodę ***get_parameters()*** .

In [14]:
solver.get_parameters()

{'t_max': 100, 'pc': 0.5, 'pm': 0.05, 'u': 250}

Teraz możemy już przejść do rozwiązania naszego problemu.

In [15]:
x_best, o_best = solver.solve(problem, pop0, printing=True)

Iteration 1: Best score = 1915, Best score to date = 1915
Iteration 2: Best score = 1916, Best score to date = 1916
Iteration 3: Best score = 1916, Best score to date = 1916
Iteration 4: Best score = 1917, Best score to date = 1917
Iteration 5: Best score = 1917, Best score to date = 1917
Iteration 6: Best score = 1917, Best score to date = 1917
Iteration 7: Best score = 1916, Best score to date = 1917
Iteration 8: Best score = 1918, Best score to date = 1918
Iteration 9: Best score = 1916, Best score to date = 1918
Iteration 10: Best score = 1919, Best score to date = 1919
Iteration 11: Best score = 1917, Best score to date = 1919
Iteration 12: Best score = 1916, Best score to date = 1919
Iteration 13: Best score = 1922, Best score to date = 1922
Iteration 14: Best score = 1919, Best score to date = 1922
Iteration 15: Best score = 1919, Best score to date = 1922
Iteration 16: Best score = 1917, Best score to date = 1922
Iteration 17: Best score = 1916, Best score to date = 1922
Iterat

In [16]:
print(f'Best score = {o_best}')

Best score = 1922


In [17]:
x_best

array([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0])

## 5. Zastosowanie algorytmu genetycznego w celu wyznaczenia najlepszego osobnika - Badanie wpływu hiperparametrów


Aby usprawnić analizę otrzymanych wyników, zaimplementowana została klasa ***GeneticAlgorithmAnalyzer***, która podczas tworzenia obiektu przyjmuje jako argumenty **solver**, **funkcję celu**, **populację początkową** oraz **liczbę przebiegów**, na podstawie których będziemy mogli ocenić dobór parametrów solvera.

Na początku przetestujemy solver z domyślnymi wartościami.

In [18]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=25)

In [19]:
analyzer.analyze()

Sample 1: Best score = 1924
Sample 2: Best score = 1923
Sample 3: Best score = 1922
Sample 4: Best score = 1925
Sample 5: Best score = 1922
Sample 6: Best score = 1926
Sample 7: Best score = 1925
Sample 8: Best score = 1924
Sample 9: Best score = 1923
Sample 10: Best score = 1924
Sample 11: Best score = 1923
Sample 12: Best score = 1926
Sample 13: Best score = 1922
Sample 14: Best score = 1923
Sample 15: Best score = 1925
Sample 16: Best score = 1923
Sample 17: Best score = 1925
Sample 18: Best score = 1922
Sample 19: Best score = 1922
Sample 20: Best score = 1924
Sample 21: Best score = 1924
Sample 22: Best score = 1924
Sample 23: Best score = 1924
Sample 24: Best score = 1922
Sample 25: Best score = 1927
 
Best sample = 25
Best score = 1927, Minimum score = 1922
Average score = 1923.76, Median score = 1924.0


Jak widać powyżej, wartość minimalna różni się od maksymalnej, natomiast średnia wyników jest zbliżona do ich mediany. Na podstawie otrzymanych wyników możemy stwierdzić, że w każdym przebiegu znalazł się osobnik, który wskazuje, że rakieta wylądowała.

Sprawdźmy jednak, czy uda nam się zredukować koszty startu rakiety tak, aby jeszcze bardziej zbliżyć się do wartości równej $2000$.

#### 5.1 Badanie wpływu prawdopodobieństwa mutacji

In [49]:
solver = GeneticAlgorithmSolver()

In [50]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [51]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1926
Sample 3: Best score = 1927
Sample 4: Best score = 1925
Sample 5: Best score = 1926
Sample 6: Best score = 1925
Sample 7: Best score = 1928
Sample 8: Best score = 1927
Sample 9: Best score = 1925
Sample 10: Best score = 1925
Sample 11: Best score = 1926
Sample 12: Best score = 1925
Sample 13: Best score = 1926
Sample 14: Best score = 1926
Sample 15: Best score = 1926
Sample 16: Best score = 1926
Sample 17: Best score = 1926
Sample 18: Best score = 1927
Sample 19: Best score = 1923
Sample 20: Best score = 1926
Sample 21: Best score = 1926
Sample 22: Best score = 1925
Sample 23: Best score = 1926
Sample 24: Best score = 1927
Sample 25: Best score = 1924
Sample 26: Best score = 1928
Sample 27: Best score = 1926
Sample 28: Best score = 1926
Sample 29: Best score = 1924
Sample 30: Best score = 1926
 
Best sample = 7
Best score = 1928, Minimum score = 1923
Average score = 1925.8, Median score = 1926.0


In [25]:
solver = GeneticAlgorithmSolver(pm=0.1)

In [26]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [27]:
analyzer.analyze()

Sample 1: Best score = 1926
Sample 2: Best score = 1927
Sample 3: Best score = 1927
Sample 4: Best score = 1927
Sample 5: Best score = 1925
Sample 6: Best score = 1925
Sample 7: Best score = 1925
Sample 8: Best score = 1926
Sample 9: Best score = 1925
Sample 10: Best score = 1926
Sample 11: Best score = 1927
Sample 12: Best score = 1929
Sample 13: Best score = 1926
Sample 14: Best score = 1927
Sample 15: Best score = 1929
Sample 16: Best score = 1927
Sample 17: Best score = 1928
Sample 18: Best score = 1925
Sample 19: Best score = 1933
Sample 20: Best score = 1924
Sample 21: Best score = 1926
Sample 22: Best score = 1925
Sample 23: Best score = 1926
Sample 24: Best score = 1926
Sample 25: Best score = 1927
Sample 26: Best score = 1925
Sample 27: Best score = 1925
Sample 28: Best score = 1929
Sample 29: Best score = 1926
Sample 30: Best score = 1924
 
Best sample = 19
Best score = 1933, Minimum score = 1924
Average score = 1926.4333333333334, Median score = 1926.0


In [37]:
solver = GeneticAlgorithmSolver(pm=0.2)

In [38]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [39]:
analyzer.analyze()

Sample 1: Best score = 1927
Sample 2: Best score = 1925
Sample 3: Best score = 1925
Sample 4: Best score = 1924
Sample 5: Best score = 1928
Sample 6: Best score = 1926
Sample 7: Best score = 1925
Sample 8: Best score = 1925
Sample 9: Best score = 1926
Sample 10: Best score = 1924
Sample 11: Best score = 1926
Sample 12: Best score = 1925
Sample 13: Best score = 1924
Sample 14: Best score = 1924
Sample 15: Best score = 1927
Sample 16: Best score = 1925
Sample 17: Best score = 1928
Sample 18: Best score = 1926
Sample 19: Best score = 1927
Sample 20: Best score = 1925
Sample 21: Best score = 1924
Sample 22: Best score = 1926
Sample 23: Best score = 1931
Sample 24: Best score = 1924
Sample 25: Best score = 1925
Sample 26: Best score = 1926
Sample 27: Best score = 1924
Sample 28: Best score = 1926
Sample 29: Best score = 1925
Sample 30: Best score = 1925
 
Best sample = 23
Best score = 1931, Minimum score = 1924
Average score = 1925.6, Median score = 1925.0


In [34]:
solver = GeneticAlgorithmSolver(pm=0.3)

In [35]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [36]:
analyzer.analyze()

Sample 1: Best score = 1924
Sample 2: Best score = 1925
Sample 3: Best score = 1925
Sample 4: Best score = 1926
Sample 5: Best score = 1925
Sample 6: Best score = 1927
Sample 7: Best score = 1924
Sample 8: Best score = 1924
Sample 9: Best score = 1924
Sample 10: Best score = 1926
Sample 11: Best score = 1928
Sample 12: Best score = 1924
Sample 13: Best score = 1924
Sample 14: Best score = 1924
Sample 15: Best score = 1924
Sample 16: Best score = 1925
Sample 17: Best score = 1924
Sample 18: Best score = 1923
Sample 19: Best score = 1925
Sample 20: Best score = 1925
Sample 21: Best score = 1926
Sample 22: Best score = 1922
Sample 23: Best score = 1924
Sample 24: Best score = 1926
Sample 25: Best score = 1926
Sample 26: Best score = 1925
Sample 27: Best score = 1926
Sample 28: Best score = 1924
Sample 29: Best score = 1928
Sample 30: Best score = 1925
 
Best sample = 11
Best score = 1928, Minimum score = 1922
Average score = 1924.9333333333334, Median score = 1925.0


In [28]:
solver = GeneticAlgorithmSolver(pm=0.5)

In [29]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [30]:
analyzer.analyze()

Sample 1: Best score = 1924
Sample 2: Best score = 1924
Sample 3: Best score = 1926
Sample 4: Best score = 1925
Sample 5: Best score = 1926
Sample 6: Best score = 1925
Sample 7: Best score = 1925
Sample 8: Best score = 1927
Sample 9: Best score = 1923
Sample 10: Best score = 1925
Sample 11: Best score = 1924
Sample 12: Best score = 1924
Sample 13: Best score = 1924
Sample 14: Best score = 1923
Sample 15: Best score = 1927
Sample 16: Best score = 1926
Sample 17: Best score = 1926
Sample 18: Best score = 1924
Sample 19: Best score = 1923
Sample 20: Best score = 1929
Sample 21: Best score = 1924
Sample 22: Best score = 1924
Sample 23: Best score = 1924
Sample 24: Best score = 1927
Sample 25: Best score = 1925
Sample 26: Best score = 1924
Sample 27: Best score = 1924
Sample 28: Best score = 1927
Sample 29: Best score = 1924
Sample 30: Best score = 1922
 
Best sample = 20
Best score = 1929, Minimum score = 1922
Average score = 1924.8333333333333, Median score = 1924.0


In [31]:
solver = GeneticAlgorithmSolver(pm=0.01)

In [32]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [33]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1924
Sample 3: Best score = 1927
Sample 4: Best score = 1925
Sample 5: Best score = 1925
Sample 6: Best score = 1926
Sample 7: Best score = 1927
Sample 8: Best score = 1926
Sample 9: Best score = 1925
Sample 10: Best score = 1924
Sample 11: Best score = 1925
Sample 12: Best score = 1925
Sample 13: Best score = 1925
Sample 14: Best score = 1925
Sample 15: Best score = 1926
Sample 16: Best score = 1925
Sample 17: Best score = 1926
Sample 18: Best score = 1929
Sample 19: Best score = 1926
Sample 20: Best score = 1925
Sample 21: Best score = 1926
Sample 22: Best score = 1925
Sample 23: Best score = 1927
Sample 24: Best score = 1925
Sample 25: Best score = 1926
Sample 26: Best score = 1925
Sample 27: Best score = 1929
Sample 28: Best score = 1926
Sample 29: Best score = 1927
Sample 30: Best score = 1928
 
Best sample = 18
Best score = 1929, Minimum score = 1924
Average score = 1925.8333333333333, Median score = 1925.5


In [46]:
solver = GeneticAlgorithmSolver(pm=0.005)

In [47]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [48]:
analyzer.analyze()

Sample 1: Best score = 1927
Sample 2: Best score = 1925
Sample 3: Best score = 1927
Sample 4: Best score = 1927
Sample 5: Best score = 1925
Sample 6: Best score = 1924
Sample 7: Best score = 1927
Sample 8: Best score = 1925
Sample 9: Best score = 1926
Sample 10: Best score = 1925
Sample 11: Best score = 1927
Sample 12: Best score = 1927
Sample 13: Best score = 1926
Sample 14: Best score = 1925
Sample 15: Best score = 1926
Sample 16: Best score = 1925
Sample 17: Best score = 1926
Sample 18: Best score = 1926
Sample 19: Best score = 1926
Sample 20: Best score = 1927
Sample 21: Best score = 1925
Sample 22: Best score = 1929
Sample 23: Best score = 1925
Sample 24: Best score = 1924
Sample 25: Best score = 1925
Sample 26: Best score = 1927
Sample 27: Best score = 1927
Sample 28: Best score = 1925
Sample 29: Best score = 1928
Sample 30: Best score = 1925
 
Best sample = 22
Best score = 1929, Minimum score = 1924
Average score = 1925.9666666666667, Median score = 1926.0


In [181]:
solver = GeneticAlgorithmSolver(pm=0.001)

In [182]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [183]:
analyzer.analyze()

Sample 1: Best score = 1923
Sample 2: Best score = 1927
Sample 3: Best score = 1927
Sample 4: Best score = 1925
Sample 5: Best score = 1925
Sample 6: Best score = 1925
Sample 7: Best score = 1926
Sample 8: Best score = 1920
Sample 9: Best score = 1921
Sample 10: Best score = 1926
Sample 11: Best score = 1924
Sample 12: Best score = 1926
Sample 13: Best score = 1923
Sample 14: Best score = 1928
Sample 15: Best score = 1924
Sample 16: Best score = 1923
Sample 17: Best score = 1923
Sample 18: Best score = 1926
Sample 19: Best score = 1926
Sample 20: Best score = 1924
Sample 21: Best score = 1927
Sample 22: Best score = 1925
Sample 23: Best score = 1927
Sample 24: Best score = 1924
Sample 25: Best score = 1923
Sample 26: Best score = 1921
Sample 27: Best score = 1927
Sample 28: Best score = 1927
Sample 29: Best score = 1924
Sample 30: Best score = 1923
 
Best sample = 14
Best score = 1928, Minimum score = 1920
Average score = 1924.6666666666667, Median score = 1925.0


In [40]:
solver = GeneticAlgorithmSolver(pm=0)

In [41]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [42]:
analyzer.analyze()

Sample 1: Best score = 1923
Sample 2: Best score = 1922
Sample 3: Best score = 1920
Sample 4: Best score = 1921
Sample 5: Best score = 1921
Sample 6: Best score = 1924
Sample 7: Best score = 1920
Sample 8: Best score = 1922
Sample 9: Best score = 1921
Sample 10: Best score = 1920
Sample 11: Best score = 1921
Sample 12: Best score = 1922
Sample 13: Best score = 1921
Sample 14: Best score = 1922
Sample 15: Best score = 1918
Sample 16: Best score = 1919
Sample 17: Best score = 1922
Sample 18: Best score = 1919
Sample 19: Best score = 1920
Sample 20: Best score = 1923
Sample 21: Best score = 1921
Sample 22: Best score = 1920
Sample 23: Best score = 1921
Sample 24: Best score = 1921
Sample 25: Best score = 1919
Sample 26: Best score = 1917
Sample 27: Best score = 1920
Sample 28: Best score = 1920
Sample 29: Best score = 1919
Sample 30: Best score = 1921
 
Best sample = 6
Best score = 1924, Minimum score = 1917
Average score = 1920.6666666666667, Median score = 1921.0


#### 5.2 Badanie wpływu prawdopodobieństwa krzyżowania

In [133]:
solver = GeneticAlgorithmSolver(pm=0, t_max=50)

In [134]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [135]:
analyzer.analyze()

Sample 1: Best score = 1919
Sample 2: Best score = 1919
Sample 3: Best score = 1923
Sample 4: Best score = 1919
Sample 5: Best score = 1920
Sample 6: Best score = 1918
Sample 7: Best score = 1919
Sample 8: Best score = 1923
Sample 9: Best score = 1919
Sample 10: Best score = 1922
Sample 11: Best score = 1918
Sample 12: Best score = 1920
Sample 13: Best score = 1921
Sample 14: Best score = 1924
Sample 15: Best score = 1919
Sample 16: Best score = 1918
Sample 17: Best score = 1917
Sample 18: Best score = 1918
Sample 19: Best score = 1919
Sample 20: Best score = 1922
Sample 21: Best score = 1919
Sample 22: Best score = 1917
Sample 23: Best score = 1919
Sample 24: Best score = 1920
Sample 25: Best score = 1917
Sample 26: Best score = 1919
Sample 27: Best score = 1921
Sample 28: Best score = 1920
Sample 29: Best score = 1919
Sample 30: Best score = 1923
 
Best sample = 14
Best score = 1924, Minimum score = 1917
Average score = 1919.7, Median score = 1919.0


In [136]:
solver = GeneticAlgorithmSolver(pm=0.1, t_max=50)

In [137]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [138]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1922
Sample 3: Best score = 1922
Sample 4: Best score = 1921
Sample 5: Best score = 1921
Sample 6: Best score = 1922
Sample 7: Best score = 1924
Sample 8: Best score = 1922
Sample 9: Best score = 1922
Sample 10: Best score = 1923
Sample 11: Best score = 1922
Sample 12: Best score = 1925
Sample 13: Best score = 1924
Sample 14: Best score = 1923
Sample 15: Best score = 1924
Sample 16: Best score = 1921
Sample 17: Best score = 1922
Sample 18: Best score = 1920
Sample 19: Best score = 1922
Sample 20: Best score = 1921
Sample 21: Best score = 1923
Sample 22: Best score = 1922
Sample 23: Best score = 1923
Sample 24: Best score = 1923
Sample 25: Best score = 1924
Sample 26: Best score = 1922
Sample 27: Best score = 1922
Sample 28: Best score = 1922
Sample 29: Best score = 1922
Sample 30: Best score = 1923
 
Best sample = 12
Best score = 1925, Minimum score = 1920
Average score = 1922.3666666666666, Median score = 1922.0


In [139]:
solver = GeneticAlgorithmSolver(pm=0.2, t_max=50)

In [140]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [141]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1921
Sample 3: Best score = 1922
Sample 4: Best score = 1920
Sample 5: Best score = 1919
Sample 6: Best score = 1920
Sample 7: Best score = 1921
Sample 8: Best score = 1920
Sample 9: Best score = 1922
Sample 10: Best score = 1921
Sample 11: Best score = 1924
Sample 12: Best score = 1919
Sample 13: Best score = 1922
Sample 14: Best score = 1925
Sample 15: Best score = 1920
Sample 16: Best score = 1922
Sample 17: Best score = 1921
Sample 18: Best score = 1922
Sample 19: Best score = 1921
Sample 20: Best score = 1922
Sample 21: Best score = 1923
Sample 22: Best score = 1925
Sample 23: Best score = 1920
Sample 24: Best score = 1921
Sample 25: Best score = 1922
Sample 26: Best score = 1921
Sample 27: Best score = 1921
Sample 28: Best score = 1924
Sample 29: Best score = 1922
Sample 30: Best score = 1922
 
Best sample = 14
Best score = 1925, Minimum score = 1919
Average score = 1921.5666666666666, Median score = 1921.5


In [142]:
solver = GeneticAlgorithmSolver(pm=0.3, t_max=50)

In [143]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [144]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1924
Sample 3: Best score = 1919
Sample 4: Best score = 1923
Sample 5: Best score = 1923
Sample 6: Best score = 1921
Sample 7: Best score = 1921
Sample 8: Best score = 1924
Sample 9: Best score = 1922
Sample 10: Best score = 1921
Sample 11: Best score = 1921
Sample 12: Best score = 1920
Sample 13: Best score = 1922
Sample 14: Best score = 1921
Sample 15: Best score = 1920
Sample 16: Best score = 1924
Sample 17: Best score = 1922
Sample 18: Best score = 1920
Sample 19: Best score = 1921
Sample 20: Best score = 1922
Sample 21: Best score = 1921
Sample 22: Best score = 1920
Sample 23: Best score = 1919
Sample 24: Best score = 1923
Sample 25: Best score = 1920
Sample 26: Best score = 1921
Sample 27: Best score = 1924
Sample 28: Best score = 1922
Sample 29: Best score = 1920
Sample 30: Best score = 1924
 
Best sample = 2
Best score = 1924, Minimum score = 1919
Average score = 1921.5666666666666, Median score = 1921.0


In [145]:
solver = GeneticAlgorithmSolver(pm=0.5, t_max=50)

In [146]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [147]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1922
Sample 3: Best score = 1920
Sample 4: Best score = 1921
Sample 5: Best score = 1922
Sample 6: Best score = 1923
Sample 7: Best score = 1925
Sample 8: Best score = 1923
Sample 9: Best score = 1925
Sample 10: Best score = 1920
Sample 11: Best score = 1922
Sample 12: Best score = 1920
Sample 13: Best score = 1920
Sample 14: Best score = 1923
Sample 15: Best score = 1919
Sample 16: Best score = 1920
Sample 17: Best score = 1922
Sample 18: Best score = 1921
Sample 19: Best score = 1921
Sample 20: Best score = 1922
Sample 21: Best score = 1920
Sample 22: Best score = 1920
Sample 23: Best score = 1920
Sample 24: Best score = 1921
Sample 25: Best score = 1923
Sample 26: Best score = 1921
Sample 27: Best score = 1922
Sample 28: Best score = 1922
Sample 29: Best score = 1924
Sample 30: Best score = 1920
 
Best sample = 7
Best score = 1925, Minimum score = 1919
Average score = 1921.5, Median score = 1921.0


In [148]:
solver = GeneticAlgorithmSolver(pm=0.9, t_max=50)

In [149]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [150]:
analyzer.analyze()

Sample 1: Best score = 1923
Sample 2: Best score = 1918
Sample 3: Best score = 1920
Sample 4: Best score = 1919
Sample 5: Best score = 1923
Sample 6: Best score = 1920
Sample 7: Best score = 1922
Sample 8: Best score = 1923
Sample 9: Best score = 1920
Sample 10: Best score = 1921
Sample 11: Best score = 1923
Sample 12: Best score = 1921
Sample 13: Best score = 1921
Sample 14: Best score = 1921
Sample 15: Best score = 1918
Sample 16: Best score = 1920
Sample 17: Best score = 1919
Sample 18: Best score = 1920
Sample 19: Best score = 1920
Sample 20: Best score = 1922
Sample 21: Best score = 1921
Sample 22: Best score = 1921
Sample 23: Best score = 1920
Sample 24: Best score = 1920
Sample 25: Best score = 1923
Sample 26: Best score = 1921
Sample 27: Best score = 1921
Sample 28: Best score = 1918
Sample 29: Best score = 1923
Sample 30: Best score = 1924
 
Best sample = 30
Best score = 1924, Minimum score = 1918
Average score = 1920.8666666666666, Median score = 1921.0


In [151]:
solver = GeneticAlgorithmSolver(t_max=50)

In [152]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [153]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1922
Sample 3: Best score = 1922
Sample 4: Best score = 1920
Sample 5: Best score = 1924
Sample 6: Best score = 1922
Sample 7: Best score = 1924
Sample 8: Best score = 1922
Sample 9: Best score = 1923
Sample 10: Best score = 1922
Sample 11: Best score = 1922
Sample 12: Best score = 1921
Sample 13: Best score = 1922
Sample 14: Best score = 1922
Sample 15: Best score = 1921
Sample 16: Best score = 1923
Sample 17: Best score = 1922
Sample 18: Best score = 1922
Sample 19: Best score = 1921
Sample 20: Best score = 1922
Sample 21: Best score = 1921
Sample 22: Best score = 1923
Sample 23: Best score = 1926
Sample 24: Best score = 1920
Sample 25: Best score = 1923
Sample 26: Best score = 1922
Sample 27: Best score = 1922
Sample 28: Best score = 1923
Sample 29: Best score = 1922
Sample 30: Best score = 1921
 
Best sample = 23
Best score = 1926, Minimum score = 1920
Average score = 1922.2333333333333, Median score = 1922.0


In [154]:
solver = GeneticAlgorithmSolver(pm=0.01, t_max=50)

In [155]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [156]:
analyzer.analyze()

Sample 1: Best score = 1924
Sample 2: Best score = 1921
Sample 3: Best score = 1923
Sample 4: Best score = 1919
Sample 5: Best score = 1919
Sample 6: Best score = 1923
Sample 7: Best score = 1922
Sample 8: Best score = 1922
Sample 9: Best score = 1919
Sample 10: Best score = 1920
Sample 11: Best score = 1920
Sample 12: Best score = 1921
Sample 13: Best score = 1920
Sample 14: Best score = 1920
Sample 15: Best score = 1921
Sample 16: Best score = 1925
Sample 17: Best score = 1921
Sample 18: Best score = 1923
Sample 19: Best score = 1921
Sample 20: Best score = 1920
Sample 21: Best score = 1921
Sample 22: Best score = 1919
Sample 23: Best score = 1923
Sample 24: Best score = 1921
Sample 25: Best score = 1919
Sample 26: Best score = 1921
Sample 27: Best score = 1921
Sample 28: Best score = 1919
Sample 29: Best score = 1919
Sample 30: Best score = 1922
 
Best sample = 16
Best score = 1925, Minimum score = 1919
Average score = 1920.9666666666667, Median score = 1921.0


In [157]:
solver = GeneticAlgorithmSolver(pm=0.005, t_max=50)

In [158]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [159]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1922
Sample 3: Best score = 1922
Sample 4: Best score = 1922
Sample 5: Best score = 1920
Sample 6: Best score = 1919
Sample 7: Best score = 1920
Sample 8: Best score = 1919
Sample 9: Best score = 1921
Sample 10: Best score = 1923
Sample 11: Best score = 1925
Sample 12: Best score = 1919
Sample 13: Best score = 1922
Sample 14: Best score = 1920
Sample 15: Best score = 1922
Sample 16: Best score = 1924
Sample 17: Best score = 1923
Sample 18: Best score = 1920
Sample 19: Best score = 1921
Sample 20: Best score = 1923
Sample 21: Best score = 1919
Sample 22: Best score = 1919
Sample 23: Best score = 1919
Sample 24: Best score = 1919
Sample 25: Best score = 1920
Sample 26: Best score = 1920
Sample 27: Best score = 1920
Sample 28: Best score = 1921
Sample 29: Best score = 1919
Sample 30: Best score = 1920
 
Best sample = 11
Best score = 1925, Minimum score = 1919
Average score = 1920.8333333333333, Median score = 1920.0


In [160]:
solver = GeneticAlgorithmSolver(pm=0.001, t_max=50)

In [161]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [162]:
analyzer.analyze()

Sample 1: Best score = 1919
Sample 2: Best score = 1920
Sample 3: Best score = 1920
Sample 4: Best score = 1919
Sample 5: Best score = 1921
Sample 6: Best score = 1920
Sample 7: Best score = 1918
Sample 8: Best score = 1920
Sample 9: Best score = 1918
Sample 10: Best score = 1919
Sample 11: Best score = 1920
Sample 12: Best score = 1920
Sample 13: Best score = 1925
Sample 14: Best score = 1920
Sample 15: Best score = 1921
Sample 16: Best score = 1917
Sample 17: Best score = 1917
Sample 18: Best score = 1920
Sample 19: Best score = 1918
Sample 20: Best score = 1920
Sample 21: Best score = 1919
Sample 22: Best score = 1921
Sample 23: Best score = 1921
Sample 24: Best score = 1919
Sample 25: Best score = 1921
Sample 26: Best score = 1918
Sample 27: Best score = 1921
Sample 28: Best score = 1920
Sample 29: Best score = 1920
Sample 30: Best score = 1920
 
Best sample = 13
Best score = 1925, Minimum score = 1917
Average score = 1919.7333333333333, Median score = 1920.0


#### 5.3 Badanie wpływu rozmiaru populacji

In [23]:
budget = 250 * 100

In [24]:
def get_n_iterations(u, budget):
    return budget/u

In [33]:
u = 5

In [34]:
population_creator = PopulationCreator(u, 200)

In [35]:
solver = GeneticAlgorithmSolver(pc=0.0, u=u, pm=0.05, t_max=get_n_iterations(u, budget))

In [36]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=25)

In [37]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1923
Sample 3: Best score = 1923
Sample 4: Best score = 1922
Sample 5: Best score = 1921


KeyboardInterrupt: 

In [100]:
solver = GeneticAlgorithmSolver(pm=0, t_max=100)

In [101]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [102]:
analyzer.analyze()

Sample 1: Best score = 1919
Sample 2: Best score = 1921
Sample 3: Best score = 1918
Sample 4: Best score = 1919
Sample 5: Best score = 1920
Sample 6: Best score = 1917
Sample 7: Best score = 1920
Sample 8: Best score = 1921
Sample 9: Best score = 1919
Sample 10: Best score = 1918
Sample 11: Best score = 1923
Sample 12: Best score = 1920
Sample 13: Best score = 1922
Sample 14: Best score = 1920
Sample 15: Best score = 1920
Sample 16: Best score = 1921
Sample 17: Best score = 1921
Sample 18: Best score = 1920
Sample 19: Best score = 1916
Sample 20: Best score = 1920
Sample 21: Best score = 1918
Sample 22: Best score = 1923
Sample 23: Best score = 1924
Sample 24: Best score = 1919
Sample 25: Best score = 1918
Sample 26: Best score = 1920
Sample 27: Best score = 1920
Sample 28: Best score = 1922
Sample 29: Best score = 1917
Sample 30: Best score = 1918
 
Best sample = 23
Best score = 1924, Minimum score = 1916
Average score = 1919.8, Median score = 1920.0


In [103]:
solver = GeneticAlgorithmSolver(pm=0.1, t_max=100)

In [104]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [105]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1921
Sample 3: Best score = 1921
Sample 4: Best score = 1922
Sample 5: Best score = 1924
Sample 6: Best score = 1922
Sample 7: Best score = 1927
Sample 8: Best score = 1923
Sample 9: Best score = 1922
Sample 10: Best score = 1922
Sample 11: Best score = 1924
Sample 12: Best score = 1925
Sample 13: Best score = 1923
Sample 14: Best score = 1922
Sample 15: Best score = 1924
Sample 16: Best score = 1924
Sample 17: Best score = 1925
Sample 18: Best score = 1924
Sample 19: Best score = 1924
Sample 20: Best score = 1924
Sample 21: Best score = 1923
Sample 22: Best score = 1922
Sample 23: Best score = 1922
Sample 24: Best score = 1923
Sample 25: Best score = 1922
Sample 26: Best score = 1922
Sample 27: Best score = 1923
Sample 28: Best score = 1924
Sample 29: Best score = 1924
Sample 30: Best score = 1922
 
Best sample = 7
Best score = 1927, Minimum score = 1921
Average score = 1923.0666666666666, Median score = 1923.0


In [106]:
solver = GeneticAlgorithmSolver(pm=0.2, t_max=100)

In [107]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [108]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1929
Sample 3: Best score = 1923
Sample 4: Best score = 1924
Sample 5: Best score = 1922
Sample 6: Best score = 1923
Sample 7: Best score = 1921
Sample 8: Best score = 1922
Sample 9: Best score = 1925
Sample 10: Best score = 1922
Sample 11: Best score = 1923
Sample 12: Best score = 1921
Sample 13: Best score = 1924
Sample 14: Best score = 1924
Sample 15: Best score = 1922
Sample 16: Best score = 1926
Sample 17: Best score = 1921
Sample 18: Best score = 1924
Sample 19: Best score = 1924
Sample 20: Best score = 1923
Sample 21: Best score = 1922
Sample 22: Best score = 1924
Sample 23: Best score = 1923
Sample 24: Best score = 1920
Sample 25: Best score = 1923
Sample 26: Best score = 1922
Sample 27: Best score = 1921
Sample 28: Best score = 1929
Sample 29: Best score = 1922
Sample 30: Best score = 1921
 
Best sample = 2
Best score = 1929, Minimum score = 1920
Average score = 1923.0333333333333, Median score = 1923.0


In [109]:
solver = GeneticAlgorithmSolver(pm=0.3, t_max=100)

In [110]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [111]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1922
Sample 3: Best score = 1923
Sample 4: Best score = 1922
Sample 5: Best score = 1922
Sample 6: Best score = 1923
Sample 7: Best score = 1922
Sample 8: Best score = 1922
Sample 9: Best score = 1921
Sample 10: Best score = 1925
Sample 11: Best score = 1921
Sample 12: Best score = 1921
Sample 13: Best score = 1921
Sample 14: Best score = 1924
Sample 15: Best score = 1923
Sample 16: Best score = 1923
Sample 17: Best score = 1925
Sample 18: Best score = 1923
Sample 19: Best score = 1922
Sample 20: Best score = 1921
Sample 21: Best score = 1924
Sample 22: Best score = 1924
Sample 23: Best score = 1922
Sample 24: Best score = 1922
Sample 25: Best score = 1922
Sample 26: Best score = 1924
Sample 27: Best score = 1921
Sample 28: Best score = 1927
Sample 29: Best score = 1923
Sample 30: Best score = 1922
 
Best sample = 28
Best score = 1927, Minimum score = 1921
Average score = 1922.6333333333334, Median score = 1922.0


In [112]:
solver = GeneticAlgorithmSolver(pm=0.5, t_max=100)

In [113]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [114]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1922
Sample 3: Best score = 1920
Sample 4: Best score = 1921
Sample 5: Best score = 1921
Sample 6: Best score = 1921
Sample 7: Best score = 1922
Sample 8: Best score = 1922
Sample 9: Best score = 1921
Sample 10: Best score = 1921
Sample 11: Best score = 1921
Sample 12: Best score = 1924
Sample 13: Best score = 1920
Sample 14: Best score = 1923
Sample 15: Best score = 1922
Sample 16: Best score = 1919
Sample 17: Best score = 1921
Sample 18: Best score = 1922
Sample 19: Best score = 1921
Sample 20: Best score = 1922
Sample 21: Best score = 1922
Sample 22: Best score = 1922
Sample 23: Best score = 1922
Sample 24: Best score = 1922
Sample 25: Best score = 1921
Sample 26: Best score = 1922
Sample 27: Best score = 1921
Sample 28: Best score = 1921
Sample 29: Best score = 1920
Sample 30: Best score = 1921
 
Best sample = 12
Best score = 1924, Minimum score = 1919
Average score = 1921.3666666666666, Median score = 1921.0


In [169]:
solver = GeneticAlgorithmSolver(pm=0.8, t_max=100)

In [170]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [171]:
analyzer.analyze()

Sample 1: Best score = 1923
Sample 2: Best score = 1923
Sample 3: Best score = 1922
Sample 4: Best score = 1921
Sample 5: Best score = 1921
Sample 6: Best score = 1923
Sample 7: Best score = 1922
Sample 8: Best score = 1922
Sample 9: Best score = 1921
Sample 10: Best score = 1921
Sample 11: Best score = 1920
Sample 12: Best score = 1920
Sample 13: Best score = 1920
Sample 14: Best score = 1921
Sample 15: Best score = 1923
Sample 16: Best score = 1922
Sample 17: Best score = 1924
Sample 18: Best score = 1922
Sample 19: Best score = 1924
Sample 20: Best score = 1920
Sample 21: Best score = 1922
Sample 22: Best score = 1920
Sample 23: Best score = 1923
Sample 24: Best score = 1925
Sample 25: Best score = 1922
Sample 26: Best score = 1920
Sample 27: Best score = 1922
Sample 28: Best score = 1923
Sample 29: Best score = 1924
Sample 30: Best score = 1921
 
Best sample = 24
Best score = 1925, Minimum score = 1920
Average score = 1921.9, Median score = 1922.0


In [130]:
solver = GeneticAlgorithmSolver(pm=0.9, t_max=100)

In [131]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [132]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1920
Sample 3: Best score = 1922
Sample 4: Best score = 1922
Sample 5: Best score = 1920
Sample 6: Best score = 1923
Sample 7: Best score = 1920
Sample 8: Best score = 1921
Sample 9: Best score = 1921
Sample 10: Best score = 1924
Sample 11: Best score = 1921
Sample 12: Best score = 1921
Sample 13: Best score = 1925
Sample 14: Best score = 1920
Sample 15: Best score = 1920
Sample 16: Best score = 1920
Sample 17: Best score = 1919
Sample 18: Best score = 1921
Sample 19: Best score = 1920
Sample 20: Best score = 1920
Sample 21: Best score = 1920
Sample 22: Best score = 1920
Sample 23: Best score = 1920
Sample 24: Best score = 1922
Sample 25: Best score = 1921
Sample 26: Best score = 1922
Sample 27: Best score = 1920
Sample 28: Best score = 1921
Sample 29: Best score = 1920
Sample 30: Best score = 1923
 
Best sample = 13
Best score = 1925, Minimum score = 1919
Average score = 1921.0333333333333, Median score = 1921.0


In [118]:
solver = GeneticAlgorithmSolver(t_max=100)

In [119]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [120]:
analyzer.analyze()

Sample 1: Best score = 1927
Sample 2: Best score = 1922
Sample 3: Best score = 1923
Sample 4: Best score = 1926
Sample 5: Best score = 1923
Sample 6: Best score = 1924
Sample 7: Best score = 1921
Sample 8: Best score = 1924
Sample 9: Best score = 1924
Sample 10: Best score = 1923
Sample 11: Best score = 1924
Sample 12: Best score = 1928
Sample 13: Best score = 1922
Sample 14: Best score = 1921
Sample 15: Best score = 1925
Sample 16: Best score = 1925
Sample 17: Best score = 1922
Sample 18: Best score = 1923
Sample 19: Best score = 1922
Sample 20: Best score = 1925
Sample 21: Best score = 1924
Sample 22: Best score = 1924
Sample 23: Best score = 1924
Sample 24: Best score = 1925
Sample 25: Best score = 1922
Sample 26: Best score = 1924
Sample 27: Best score = 1924
Sample 28: Best score = 1922
Sample 29: Best score = 1928
Sample 30: Best score = 1925
 
Best sample = 12
Best score = 1928, Minimum score = 1921
Average score = 1923.8666666666666, Median score = 1924.0


In [121]:
solver = GeneticAlgorithmSolver(pm=0.01, t_max=100)

In [122]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [123]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1923
Sample 3: Best score = 1923
Sample 4: Best score = 1923
Sample 5: Best score = 1921
Sample 6: Best score = 1921
Sample 7: Best score = 1921
Sample 8: Best score = 1925
Sample 9: Best score = 1924
Sample 10: Best score = 1923
Sample 11: Best score = 1921
Sample 12: Best score = 1923
Sample 13: Best score = 1924
Sample 14: Best score = 1923
Sample 15: Best score = 1923
Sample 16: Best score = 1923
Sample 17: Best score = 1922
Sample 18: Best score = 1924
Sample 19: Best score = 1922
Sample 20: Best score = 1923
Sample 21: Best score = 1922
Sample 22: Best score = 1925
Sample 23: Best score = 1922
Sample 24: Best score = 1923
Sample 25: Best score = 1923
Sample 26: Best score = 1924
Sample 27: Best score = 1924
Sample 28: Best score = 1923
Sample 29: Best score = 1924
Sample 30: Best score = 1920
 
Best sample = 1
Best score = 1925, Minimum score = 1920
Average score = 1922.9, Median score = 1923.0


In [124]:
solver = GeneticAlgorithmSolver(pm=0.005, t_max=100)

In [125]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [126]:
analyzer.analyze()

Sample 1: Best score = 1920
Sample 2: Best score = 1924
Sample 3: Best score = 1921
Sample 4: Best score = 1926
Sample 5: Best score = 1922
Sample 6: Best score = 1923
Sample 7: Best score = 1922
Sample 8: Best score = 1924
Sample 9: Best score = 1924
Sample 10: Best score = 1923
Sample 11: Best score = 1920
Sample 12: Best score = 1923
Sample 13: Best score = 1920
Sample 14: Best score = 1923
Sample 15: Best score = 1922
Sample 16: Best score = 1921
Sample 17: Best score = 1921
Sample 18: Best score = 1922
Sample 19: Best score = 1923
Sample 20: Best score = 1923
Sample 21: Best score = 1919
Sample 22: Best score = 1921
Sample 23: Best score = 1923
Sample 24: Best score = 1924
Sample 25: Best score = 1921
Sample 26: Best score = 1922
Sample 27: Best score = 1923
Sample 28: Best score = 1922
Sample 29: Best score = 1921
Sample 30: Best score = 1921
 
Best sample = 4
Best score = 1926, Minimum score = 1919
Average score = 1922.1333333333334, Median score = 1922.0


In [127]:
solver = GeneticAlgorithmSolver(pm=0.001, t_max=100)

In [128]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [129]:
analyzer.analyze()

Sample 1: Best score = 1919
Sample 2: Best score = 1923
Sample 3: Best score = 1921
Sample 4: Best score = 1919
Sample 5: Best score = 1921
Sample 6: Best score = 1920
Sample 7: Best score = 1920
Sample 8: Best score = 1922
Sample 9: Best score = 1919
Sample 10: Best score = 1922
Sample 11: Best score = 1919
Sample 12: Best score = 1921
Sample 13: Best score = 1921
Sample 14: Best score = 1919
Sample 15: Best score = 1922
Sample 16: Best score = 1920
Sample 17: Best score = 1919
Sample 18: Best score = 1922
Sample 19: Best score = 1925
Sample 20: Best score = 1918
Sample 21: Best score = 1922
Sample 22: Best score = 1921
Sample 23: Best score = 1918
Sample 24: Best score = 1921
Sample 25: Best score = 1920
Sample 26: Best score = 1921
Sample 27: Best score = 1922
Sample 28: Best score = 1921
Sample 29: Best score = 1923
Sample 30: Best score = 1920
 
Best sample = 19
Best score = 1925, Minimum score = 1918
Average score = 1920.7, Median score = 1921.0
